This is where we are going to figure out how to build a ref data base that updates every night based and stores as a CSV file somewhere - it will have ref data for all categories 

In [1]:
# importing the things we might need

import pandas as pd 
import os
import requests
from dotenv import load_dotenv


load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
TENANT_URL = os.getenv("TENANT_URL")
PROFILE_URL = os.getenv("PROFILE_URL")
FORCEDECKS_URL = os.getenv("FORCEDECKS_URL")
DYNAMO_URL = os.getenv("DYNAMO_URL")
SMARTSPEED_URL = os.getenv("SMARTSPEED_URL")
AUTH_URL = os.getenv("AUTH_URL")
TENANT_ID = os.getenv("TENANT_ID")



In [2]:
# putting the access token function here just to make things easy
def get_access_token(CLIENT_ID, CLIENT_SECRET, AUTH_URL):
    payload = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }

    auth_response = requests.post(AUTH_URL, data=payload)

    if auth_response.status_code == 200:
        access_token = auth_response.json()["access_token"]
        print("Access Token Generated")
        return access_token
    else:
        print("Auth Failed:", auth_response.status_code)
        print(auth_response.text)
        return None

# Generating Access Token
token = get_access_token(CLIENT_ID, CLIENT_SECRET, AUTH_URL)


Access Token Generated


Access tokens expire. Every time we run this code we get a new token to use saved as "token"

If we every want to access the profiles or tests then we have to use the token to do that

chat suggested setting a timmer to generate a new one every time it expires but we can worry about something like that later

In [7]:
# get profile data

from datetime import datetime

today = datetime.today()

def get_profiles(token, TENANT_ID, PROFILE_URL):
    url=f"{PROFILE_URL}/profiles?tenantId={TENANT_ID}"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        print("Profiles found")
        df = pd.DataFrame(response.json()['profiles'])
        df['givenName'] = df['givenName'].str.strip()
        df['familyName'] = df['familyName'].str.strip()
        df['fullName'] = df['givenName'] + ' ' + df['familyName']
        df['dateOfBirth'] = pd.to_datetime(df['dateOfBirth'])
        df['age'] = today.year - df['dateOfBirth'].dt.year - (
            (today.month < df['dateOfBirth'].dt.month) | 
            ((today.month == df['dateOfBirth'].dt.month) & (today.day < df['dateOfBirth'].dt.day)))           .astype(int)
        return df
    else:
        print(f"Failed to get profiles: {response.status_code}")

get_profiles(token = token, TENANT_ID=TENANT_ID, PROFILE_URL=PROFILE_URL)

Profiles found


,profileId,syncId,givenName,familyName,dateOfBirth,externalId,fullName,age
0,36a6daee-ced1-41d0-857d-00045e0a4f99,None,Danny,Herrington,2004-01-29,None,Danny Herrington,21
1,0facee79-a342-4fb9-af37-001402c30150,None,Jack,Winkler,1998-11-04,None,Jack Winkler,26
2,db8af672-bc03-419c-bf76-0025d35a82f7,None,Jaxon,Walker,2009-12-12,None,Jaxon Walker,15
3,251efcda-07cb-4699-8339-00310dcbb1af,None,James,McArthur,1996-12-11,None,James McArthur,28
4,c19ab400-5cef-4875-ac6c-003315e5ca4b,None,Kyler,Coulson,2007-05-13,None,Kyler Coulson,18
...,...,...,...,...,...,...,...,...
1919,f11c0943-041d-4d04-9669-ff429e5e6f8a,None,Rydyr,Foegen,2024-04-05,None,Rydyr Foegen,1
1920,876a649b-6f29-4c97-8c03-ff565bcb1423,None,Jack,Jaworsky,2025-01-31,None,Jack Jaworsky,0
1921,1b5f447e-e857-47d5-9b0a-ff6cba5c1203,None,Luke,Reasbeck,2007-03-23,None,Luke Reasbeck,18
1922,dc90163d-347e-4ba0-8cde-ff6fe572012b,None,Amy,Xia,2008-03-26,None,Amy Xia,17


In [4]:
# get force decks tests - I don't really understand what it means when it says tests though

def force_decks_test(FORCEDECKS_URL, TENANT_ID, DATE, token):
    url=f"{FORCEDECKS_URL}/tests?TenantId={TENANT_ID}&ModifiedFromUtc={DATE}"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        df = response.json()['tests']
        tests = pd.DataFrame(df)
        tests.to_csv('../Outputs/Tests.csv')
        print(response.status_code)
        return
    else:
        print(response.status_code)

date = "2025-06-24T00:00:00Z"
 
tests = force_decks_test(FORCEDECKS_URL=FORCEDECKS_URL, TENANT_ID=TENANT_ID, DATE=date, token=token)

200


In [5]:
# getting result definitions as a csv file

def FD_results(FORCEDECKS_URL):
    url=f"{FORCEDECKS_URL}/resultdefinitions"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers = headers)

    if response.status_code == 200:
        df = response.json()['resultDefinitions']
        results = pd.DataFrame(df)
        print("Result def returned")
        results.to_csv('../Outputs/ResultDefinitions.csv')
        return
    else:
        print(response.status_code)


resultsDef = FD_results(FORCEDECKS_URL=FORCEDECKS_URL)


Result def returned


so we figure out how to pull an actual test from the api - we need the tenant id which is the same as the teamID and we need a test ID which is pulled using the force_decks_tests function. I am going to use the force_decks_test function to pull that kid that jumped this mornings (Hunter Gotschall) data now

In [8]:
#Start by getting Hunter G's profile ID

import tkinter as tk 
from tkinter import ttk

profiles = get_profiles(token=token, TENANT_ID=TENANT_ID, PROFILE_URL=PROFILE_URL)

name_to_id = dict(zip(profiles['fullName'], profiles['profileId']))
name_list = sorted(name_to_id.keys())

root = tk.Tk()
root.title("VALD Profile ID Finder")

def update_listbox(*args):
    typed = name_var.get().lower()
    listbox.delete(0, tk.END)
    if typed:
        matches = [name for name in name_list if typed in name.lower()]
        for name in matches:
            listbox.insert(tk.END, name)

def fill_entry(event):
    selected_name = listbox.get(listbox.curselection())
    name_var.set(selected_name)
    listbox.delete(0, tk.END)
    search_profile()


def search_profile():
    input_name = name_var.get().strip()
    profile_id = name_to_id.get(input_name)
    if profile_id:
        result_label.config(text=f"Profile ID: {profile_id}")
    else:
        result_label.config(text="Name not found.")


tk.Label(root, text="Start typing a name:").pack(padx=10, pady=(10, 0))

name_var = tk.StringVar()
name_var.trace("w", update_listbox)

entry = tk.Entry(root, textvariable=name_var, width=40)
entry.pack(padx=10, pady=5)

listbox = tk.Listbox(root, height=6, width=40)
listbox.pack(padx=10, pady=(0, 10))
listbox.bind("<<ListboxSelect>>", fill_entry)

result_label = tk.Label(root, text="")
result_label.pack(padx=10, pady=(0, 10))

root.mainloop()

Profiles found


In [ ]:
def force_decks_test_profile(FORCEDECKS_URL, TENANT_ID, DATE, profileId, token):
    url=f"{FORCEDECKS_URL}/tests?TenantId={TENANT_ID}&ModifiedFromUtc={DATE}&ProfileId={profileId}"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        df = response.json()
        print(response.status_code)
        return df
    else:
        print(response.status_code)




Things I want from a CMJ (Result_key)
- BODY_WEIGHT (655386) (Trial)
- BODY_WEIGHT_LBS (65587) (Trial)
- COUNTERMOVEMENT_DEPTH (6553603) (Trial)
-BODYMASS_RELATIVE_TAKEOFF_POWER (6553604) (Trial)
-JUMP_HEIGHT_INCHES_IMP_MOM (6553614) (Trial)
-MEAN_TAKEOFF_FORCE (6553619) (Trial)
-MEAN_TAKEOFF_FORCE (6553619) (Asym)
-MEAN_TAKEOFF_FORCE (6553620) (Trial)
-MEAN_TAKEOFF_FORCE (6553620) (Asym)
-PEAK_TAKEOFF_FORCE (6553632) (Trial)
-PEAK_TAKEOFF_POWER (6553635) (Trial)
-CONCENTRIC_DURATION (6553657) (Trial)








